# Modelamiento no supervisado en base a tópicos

In [1]:
import sys
sys.path.insert(0, '..')
from utils.preprocesamiento import StemmerTokenizer

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

In [2]:
from utils.cargar import df_caso
from utils.preprocesamiento import process_df

caso = 'julieta'
df = df_caso(caso)

df = process_df(df,'comment','sel',verbose=True)

df = df.drop(columns=['user_id','team_id','gender','df','title','opt_left','opt_right','max_num','phase','time','curso'])

34 rows found with non string elements for column comment (0.67%)
Deleting 255 columns for which max target value is over 7 (5.00%)
4806 available rows after processing


In [3]:
from sklearn.model_selection import train_test_split

df_train, df_test, _, _ = train_test_split(df, df['sel'], test_size=.05, stratify=df['sel'], random_state=0)

In [4]:
tokenized_corpus = [tokenizador(document) for document in df_train['comment']]
tokenized_test = [tokenizador(document) for document in df_test['comment']]

In [5]:
import gensim
from gensim import corpora

# Create a dictionary from the tokenized corpus
dictionary = corpora.Dictionary(tokenized_corpus)

# Convert the tokenized corpus into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_corpus]

## Número óptimo de tópicos

In [6]:
from gensim.models import CoherenceModel

# Set the range of topic numbers to try
min_topics = 2
max_topics = 10
step_size = 1

# Initialize variables for best coherence score and best number of topics
best_coherence_score = -1
best_num_topics = -1

# Iterate over the range of topic numbers
for num_topics in range(min_topics, max_topics+1, step_size):
    # Train the LDA model
    lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=num_topics, id2word=dictionary, passes=10)
    
    # Calculate coherence score
    coherence_model = CoherenceModel(model=lda_model, texts=tokenized_corpus, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    
    # Check if coherence score is the best so far
    if coherence_score > best_coherence_score:
        best_coherence_score = coherence_score
        best_num_topics = num_topics

# Print the best number of topics
print(f"Best number of topics: {best_num_topics}")

Best number of topics: 2


In [7]:
%%time
lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=best_num_topics, id2word=dictionary, passes=10)

CPU times: user 11.4 s, sys: 4.05 ms, total: 11.4 s
Wall time: 11.4 s


In [8]:
# Print the generated topics
topics = lda_model.print_topics(num_topics=num_topics)
for topic in topics:
    print(topic)

(0, '0.022*"beca" + 0.021*"si" + 0.017*"copiar" + 0.015*"Julieta" + 0.010*"puede" + 0.010*"debería" + 0.008*"ser" + 0.008*"futuro" + 0.007*"situación" + 0.006*"tener"')
(1, '0.034*"información" + 0.032*"si" + 0.019*"usar" + 0.018*"beca" + 0.017*"Julieta" + 0.017*"debería" + 0.013*"respuesta" + 0.013*"control" + 0.011*"grupo" + 0.011*"podría"')


In [9]:
print(' '.join(tokenized_test[0]) + '\n')

# Convert the tokenized document into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(tokenized_test[0])]

# Get the topic probabilities for the new document
topic_probs = lda_model.get_document_topics(doc_term_matrix)[0]

# Print the topic probabilities
for topic, prob in topic_probs:
    print(f"Topic {topic}: {prob}")

No deberia utilizarla forma obtendra nota probable refleje aprendizaje perjudicara hora aprender contenidos futuros

Topic 0: 0.30374690890312195
Topic 1: 0.6962531208992004


## Visualización

In [10]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
pyLDAvis.display(vis_data)